# Command Pattern

Este patron convierte una solicitud en un objeto independiente que contiene toda la información sobre la solicitud. Esta transformación permite parametrizar métodos con diferentes solicitudes, retrasar o poner en cola la ejecución de una solicitud y respaldar operaciones que no se pueden deshacer

## Caso de Estudio

Se requiere un sistema tal que los meseros tomen los pedidos y la cocina eejcute las órdenes de los pedidos

In [1]:
from abc import ABC, abstractmethod

class Orden(ABC):
    @abstractmethod
    def ejecutar(self):
        pass

# Se implementan clases concretas para las ordenes a utilizar
class OrdenCarne(Orden):
    def __init__(self, cocina):
        self.cocina = cocina

    def ejecutar(self):
        self.cocina.preparar_carne()

class OrdenHelado(Orden):
    def __init__(self, cocina):
        self.cocina = cocina

    def ejecutar(self):
        self.cocina.preparar_helado()

# Se define el receptor de alimentos para la cocina
class Cocina:
    def preparar_carne(self):
        print("Preparando carne")

    def preparar_helado(self):
        print("Preparando helado")

# Se define un invocador para el almacenamienteo, limpeza, envio  y recibimiento de ordenes
class Mesero:
    def __init__(self):
        self.ordenes = []

    def tomar_orden(self, orden):
        self.ordenes.append(orden)

    def enviar_ordenes(self):
        print("Enviando órdenes a la cocina")
        for orden in self.ordenes:
            orden.ejecutar()
        self.ordenes = []

#Ejecucion del proceso al clinete

if __name__ == "__main__":
    cocina = Cocina()
    orden_carne = OrdenCarne(cocina)
    orden_helado = OrdenHelado(cocina)

    mesero = Mesero()
    mesero.tomar_orden(orden_carne)
    mesero.tomar_orden(orden_helado)
    mesero.enviar_ordenes()

Enviando órdenes a la cocina
Preparando carne
Preparando helado


El modelo planteado satisface los principios SOLID:

    S - Principio de Responsabilidad Única (SRP)
        Cada clase tiene una única responsabilidad:
        OrdenCarne y OrdenHelado encapsulan la acción específica de cada orden.
        Cocina se encarga de preparar los alimentos.
        Mesero gestiona, almacena y envía las órdenes sin conocer los detalles de su ejecución.

    O - Principio de Abierto/Cerrado (OCP)
        Se pueden agregar nuevas órdenes (por ejemplo, OrdenPizza) sin modificar el código existente, solo extendiendo la abstracción Orden.

    L - Principio de Sustitución de Liskov (LSP)
        Todas las clases que heredan de Orden (como OrdenCarne y OrdenHelado) pueden ser usadas de forma intercambiable sin afectar el funcionamiento del sistema.

    I - Principio de Segregación de Interfaces (ISP)
        La interfaz definida en Orden es mínima (solo el método ejecutar), lo que obliga a las clases a implementar únicamente lo necesario.

    D - Principio de Inversión de Dependencias (DIP)
        Mesero depende de la abstracción Orden en lugar de depender de implementaciones concretas, facilitando la extensión y el mantenimiento del código.

Adicionalmente se satisfacen los siguientes principios GRASP:

    1 - Polimorfismo
        Se utiliza polimorfismo, ya que todas las órdenes implementan la interfaz Orden y pueden ser tratadas de manera uniforme por Mesero.

    2 - Bajo Acoplamiento
        El uso del patrón Command desacopla a Mesero de Cocina, ya que el mesero solo interactúa con la abstracción Orden, sin conocer detalles de la ejecución.

    3 - Indirection
        Los comandos actúan como intermediarios entre el invocador (Mesero) y el receptor (Cocina), facilitando la delegación y extensión de funcionalidades.

    4 - Alta Cohesión
        Cada clase se concentra en una tarea específica, lo que simplifica la comprensión y el mantenimiento del sistema.

# Mediator
El patrón restringe las comunicaciones directas entre los objetos, forzándolos a colaborar únicamente a través de un objeto mediador.

## Sala de chat
Se necesita implementar una sala de chat donde los participantes no se envían mensajes directamente entre sí, sino que todos los mensajes pasan por el grupo.

In [3]:
from abc import ABC, abstractmethod

# Interfaz del Mediador
class ChatMediator(ABC):
    @abstractmethod
    def send_message(self, message: str, user):
        pass

# Implementación del Mediador
class ChatRoom(ChatMediator):
    def __init__(self):
        self.users = []

    def add_user(self, user):
        self.users.append(user)

    def send_message(self, message: str, user):
        for u in self.users:
            if u != user:  # No enviarse mensajes a sí mismo
                u.receive(message)

# Clase Colega (Usuario)
class User:
    def __init__(self, name: str, mediator: ChatMediator):
        self.name = name
        self.mediator = mediator
        self.mediator.add_user(self)

    def send(self, message: str):
        print(f"{self.name} envía: {message}")
        self.mediator.send_message(message, self)

    def receive(self, message: str):
        print(f"{self.name} recibe: {message}")

# Caso de uso
chat = ChatRoom()

user1 = User("Alice", chat)
user2 = User("Bob", chat)
user3 = User("Charlie", chat)
user1.send("Hola a todos!")
user2.send("Hola Alice!")
user3.send("Buenas!")


Alice envía: Hola a todos!
Bob recibe: Hola a todos!
Charlie recibe: Hola a todos!
Bob envía: Hola Alice!
Alice recibe: Hola Alice!
Charlie recibe: Hola Alice!
Charlie envía: Buenas!
Alice recibe: Buenas!
Bob recibe: Buenas!


El patron multiton cumple algunos principios SOLID

    S - Principio de Responsabilidad Única (SRP)
        ChatRoom solo se encarga de la mediación de mensajes.
        User solo representa a un usuario y maneja el envío/recepción de mensajes.

    O - Principio de Abierto/Cerrado (OCP)
        Se pueden agregar nuevas funcionalidades al chat (como mensajes privados) sin modificar las clases existentes.

    L - Principio de Sustitución de Liskov (LSP)
        ChatRoom puede ser reemplazado por otra implementación de ChatMediator sin afectar a User.

    I - Principio de Segregación de Interfaces (ISP)
        ChatMediator solo define los métodos que necesita (send_message).
        No fuerza a sus implementaciones a definir métodos innecesarios.

    D - Principio de Inversión de Dependencias (DIP)
        User no depende de una implementación concreta de ChatMediator, sino de su abstracción, lo que permite cambiar fácilmente la lógica del mediador.

Adicionalmente se satisfacen los siguientes principios GRASP:

    1 - Indirection
        ChatRoom actúa como un intermediario, evitando que los usuarios interactúen directamente.

    2 - Bajo Acoplamiento
        Los usuarios no dependen entre sí, sino solo del mediador.

    3 - Alta Cohesión
        Cada clase tiene un propósito bien definido:
        User representa a un usuario.
        ChatRoom se encarga de la comunicación.